In [ ]:
import tensorflow as tf
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test= np.load("/content/drive/Shareddrives/2022동계우수학부생_DSAIL/chest_xray_data_rgb.npy", allow_pickle=True)

In [ ]:
x = np.load("/content/drive/Shareddrives/2022동계우수학부생_DSAIL/random_sampling_x.npy",allow_pickle=True)
y = np.load("/content/drive/Shareddrives/2022동계우수학부생_DSAIL/random_sampling_y.npy", allow_pickle=True)

In [ ]:
x.shape

(5, 3912, 224, 224, 3)

In [ ]:
x_1, y_1 = x[0],y[0]
x_2, y_2 = x[1],y[1]
x_3, y_3 = x[2],y[2]
x_4, y_4 = x[3],y[3]
x_5, y_5 = x[4],y[4]

In [ ]:
x_1.shape

(3912, 224, 224, 3)

In [ ]:
vgg_19 = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=X_train[0].shape,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

vgg_19.trainable = True

vgg_19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
for layer in vgg_19.layers[:-4]:
   layer.trainable = False

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(vgg_19.output)
x = tf.keras.layers.Flatten()(vgg_19.output)
# x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(vgg_19.input, x)

In [ ]:
# augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 30,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 fill_mode='nearest')

train_generator = datagen.flow(x_1, y_1,batch_size=32,shuffle=True,seed=42)
valid_generator = datagen.flow(X_val,y_val,batch_size=32,seed=42)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),loss='binary_crossentropy', metrics=['accuracy'])
# history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_val, y_val))
history = model.fit(train_generator,epochs=15, validation_data=valid_generator) # augmentation 적용

results = model.evaluate(X_test, y_test)
print(results)

Epoch 1/15
 56/123 [============>.................] - ETA: 24:21 - loss: 5.1589 - accuracy: 0.7076

KeyboardInterrupt: ignored

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred.reshape(624)
y_pred1 = np.zeros(624)
for i in range(len(y_pred)):
  if y_pred[i] > 0.5:
    y_pred1[i]=1


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
target_names = ['NORMAL', 'PNEUMONIA']
print(confusion_matrix(y_test, y_pred1))
print(classification_report(y_test, y_pred1, target_names=target_names))

In [ ]:
model.save('vgg19_1')